In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
import os
import pandas as pd
import numpy as np
from torch import nn
from transformers import AutoTokenizer, AutoModelForPreTraining, AutoModelForMaskedLM, pipeline, Trainer, DataCollatorForLanguageModeling, TrainingArguments
from tqdm.auto import tqdm
from datasets import load_dataset, Dataset
import datasets

WORKING_DIR = r'C:\Users\Furkan\Desktop\study\nlp\teknofest\notebooks'

os.chdir(WORKING_DIR)

ds = load_dataset('csv', data_files='data_text.csv', split='train')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-uncased')
model = AutoModelForMaskedLM.from_pretrained('dbmdz/bert-base-turkish-uncased', output_hidden_states=True).to(device)

In [ ]:
def encode(examples):
    return tokenizer(examples['data_text'], truncation=True, padding='max_length')

ds = ds.map(encode, batched=True)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
args = TrainingArguments(output_dir="tmp_trainer", do_train=True, do_eval=False, learning_rate=1e-05, per_device_train_batch_size=8)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

trainer = Trainer(model=model, args=args, tokenizer=tokenizer, train_dataset=ds, data_collator=collator)
trainer.train()

In [ ]:
trainer.save_model("trainer_output")